In [1]:
import numpy as np
from io import TextIOWrapper

ang2au = 1.8897259886

In [2]:
file = "F1_2_0/Febpy_1S_MCSCF.out"

fname = open(file, 'r')

if isinstance(fname, TextIOWrapper):
    flines = fname.readlines()

In [3]:
COORD_SWITCH = 0

BASIS_SWITCH = 0
BASIS_ATOM_NUMER = 0

NATORB_SWITCH = 0
OPTORB_SWITCH = 0

COORDS = []

for i,line in enumerate(flines):
    thisline = line.split()
    
    if thisline == ["CHARGE", "X", "Y", "Z"]:
        COORD_SWITCH = 1
        continue
        
    if COORD_SWITCH:
        if thisline == []:
            COORD_SWITCH = 0
            continue
        COORDS.append(thisline) ## In Bohr
        
    if thisline == ["SHELL", "TYPE", "PRIMITIVE", "EXPONENT", "CONTRACTION", "COEFFICIENT(S)"]:
        BASIS_SWITCH = 1
        continue
        
    if COORD_SWITCH:
        if thisline == []:
            continue
        if len(thisline) == 1:
            BASIS_ATOM_NUMER += 1
            
        COORDS.append(thisline) ## In Bohr

In [4]:
COORDS

[['FE', '26.0', '0.0000000000', '0.0000000000', '2.0177479881'],
 ['FE', '26.0', '0.0000000000', '0.0000000000', '2.0177479881'],
 ['N', '7.0', '-0.0000000000', '-2.3899513099', '-0.7743009938'],
 ['N', '7.0', '-0.0000000000', '-2.3899513099', '-0.7743009938'],
 ['N', '7.0', '0.0000000000', '2.3899513099', '-0.7743009938'],
 ['N', '7.0', '0.0000000000', '2.3899513099', '-0.7743009938'],
 ['C', '6.0', '-0.0000000000', '-1.3694294323', '-3.1452875996'],
 ['C', '6.0', '-0.0000000000', '-1.3694294323', '-3.1452875996'],
 ['C', '6.0', '0.0000000000', '1.3694294323', '-3.1452875996'],
 ['C', '6.0', '0.0000000000', '1.3694294323', '-3.1452875996'],
 ['C', '6.0', '-0.0000000000', '-4.9360571789', '-0.5807584329'],
 ['C', '6.0', '-0.0000000000', '-4.9360571789', '-0.5807584329'],
 ['C', '6.0', '0.0000000000', '4.9360571789', '-0.5807584329'],
 ['C', '6.0', '0.0000000000', '4.9360571789', '-0.5807584329'],
 ['C', '6.0', '-0.0000000000', '-6.5172507226', '-2.6579018883'],
 ['C', '6.0', '-0.000000

In [116]:
NATORB_SWITCH = 0
OPTORB_SWITCH = 0

for i,line in enumerate(flines):
    thisline = line.split()
    
    if "NUMBER OF CORE MOLECULAR ORBITALS" in line:
        nCOs = int(thisline[-1])
        continue
    
    if "NUMBER OF OCCUPIED MOLECULAR ORBITALS" in line:
        nOOs = int(thisline[-1])
        continue

    if "TOTAL NUMBER OF MOLECULAR ORBITALS" in line:
        nMOs = int(thisline[-1])
        continue
        
    if "TOTAL NUMBER OF ATOMIC ORBITALS" in line:
        nAOs = int(thisline[-1])
        natorbs = np.zeros((nOOs, nAOs))
        optorbs = np.zeros((nMOs, nAOs))
        continue
        
    if "MCSCF NATURAL ORBITALS" in line:
        NAT_IND = i
        continue
        
    if "MCSCF OPTIMIZED ORBITALS" in line:
        OPT_IND = i
        break
        
    

In [117]:
AO_INFO = []

for i in range(nAOs):
    AO_INFO.append(flines[NAT_IND+6+i].split()[:4])

In [118]:
NATMOS = []
NATEIGS = []

for i in range(nOOs):
    MO = []
    NATEIGS.append(flines[NAT_IND + 4 + ((i//5)*(nAOs+4))].split()[(i%5)])
    for j in range(nAOs):
        MO.append(flines[NAT_IND + 6 + ((i//5)*(nAOs+4)) + j].split()[4+(i%5)])
    NATMOS.append(MO)
    



In [119]:
OPTMOS = []
OPTEIGS = []

for i in range(nMOs):
    MO = []
    OPTEIGS.append(flines[OPT_IND + 4 + ((i//5)*(nAOs+4))].split()[(i%5)])
    for j in range(nAOs):
        MO.append(flines[OPT_IND + 6 + ((i//5)*(nAOs+4)) + j].split()[4+(i%5)])
    OPTMOS.append(MO)

In [127]:
f = open("F1_2_0/nat_app_molden.txt", "w")
f.write("[MO]\n")
for i in range(nOOs):

    if i < nCOs:
        f.write(" Ene= {}\n".format(NATEIGS[i]))
    else:
        f.write(" Ene= 0\n")
        
    f.write(" Spin= Alpha\n")
    
    if i < nCOs:
        f.write(" Occup=   2.000000\n")
    else:
        f.write(" Occup=   {}\n".format(NATEIGS[i]))
        
    for j in range(nAOs):
        f.write(" {}  {}\n".format(j+1, NATMOS[i][j]))
        
f.close()

In [129]:
f = open("F1_2_0/opt_app_molden.txt", "w")
f.write("[MO]\n")
for i in range(nMOs):

    f.write(" Ene= {}\n".format(OPTEIGS[i]))
        
    f.write(" Spin= Alpha\n")
    
    if i < nCOs:
        f.write(" Occup=   2.000000\n")
    else:
        f.write(" Occup=   0\n".format(OPTEIGS[i]))
        
    for j in range(nAOs):
        f.write(" {}  {}\n".format(j+1, OPTMOS[i][j]))
        
f.close()